In [1]:
import pandas as pd

In [8]:
df = pd.read_csv('/Users/yuyingshan/Desktop/complaints.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1868263 entries, 0 to 1868262
Data columns (total 18 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   Date received                 object
 1   Product                       object
 2   Sub-product                   object
 3   Issue                         object
 4   Sub-issue                     object
 5   Consumer complaint narrative  object
 6   Company public response       object
 7   Company                       object
 8   State                         object
 9   ZIP code                      object
 10  Tags                          object
 11  Consumer consent provided?    object
 12  Submitted via                 object
 13  Date sent to company          object
 14  Company response to consumer  object
 15  Timely response?              object
 16  Consumer disputed?            object
 17  Complaint ID                  int64 
dtypes: int64(1), object(17)
memory usage: 256.

In [ ]:
# source: https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17 

# dataset from https://catalog.data.gov/dataset/consumer-complaint-database

In [9]:
df.Product.value_counts()

Credit reporting, credit repair services, or other personal consumer reports    573588
Debt collection                                                                 324543
Mortgage                                                                        315552
Credit reporting                                                                140432
Credit card or prepaid card                                                      96025
Credit card                                                                      89190
Bank account or service                                                          86206
Checking or savings account                                                      77414
Student loan                                                                     59916
Consumer Loan                                                                    31604
Vehicle loan or lease                                                            21300
Money transfer, virtual currency, or money 

In [10]:
df.isnull().values.any()

df.shape

(1868263, 18)

In [11]:
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2019-09-24,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,transworld systems inc. \nis trying to collect...,NaN,TRANSWORLD SYSTEMS INC,FL,335XX,NaN,Consent provided,Web,2019-09-24,Closed with explanation,Yes,NaN,3384392
1,2019-09-19,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,PA,15206,NaN,Consent not provided,Web,2019-09-20,Closed with non-monetary relief,Yes,NaN,3379500
2,2019-11-08,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,"Over the past 2 weeks, I have been receiving e...",NaN,"Diversified Consultants, Inc.",NC,275XX,NaN,Consent provided,Web,2019-11-08,Closed with explanation,Yes,NaN,3433198
3,2019-09-15,Debt collection,Other debt,Attempts to collect debt not owed,Debt was result of identity theft,Pioneer has committed several federal violatio...,NaN,Pioneer Capital Solutions Inc,CA,925XX,NaN,Consent provided,Web,2019-09-15,Closed with explanation,Yes,NaN,3374555
4,2019-05-20,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,NC,28078,NaN,Consent not provided,Web,2019-05-20,Closed with non-monetary relief,Yes,NaN,3247381


In [12]:
# Label Consolidation

df.loc[df['Product'] == 'Credit reporting', 'Product'] = 'Credit reporting, credit repair services, or other personal consumer reports'
df.loc[df['Product'] == 'Credit card', 'Product'] = 'Credit card or prepaid card'
df.loc[df['Product'] == 'Payday loan', 'Product'] = 'Payday loan, title loan, or personal loan'
df.loc[df['Product'] == 'Virtual currency', 'Product'] = 'Money transfer, virtual currency, or money service'
df = df[df.Product != 'Other financial service']

In [15]:
pip install cufflinks

     |████████████████████████████████| 81 kB 2.5 MB/s eta 0:00:011
     |████████████████████████████████| 13.2 MB 16.3 MB/s eta 0:00:01
  Created wheel for cufflinks: filename=cufflinks-0.17.3-py3-none-any.whl size=67921 sha256=1fdb49f6e9371fc53582e19a517bd2f364f905afb0340911ef0a6d1d7ba8329b
  Stored in directory: /Users/yuyingshan/Library/Caches/pip/wheels/e1/27/13/3fe67fa7ea7be444b831d117220b3b586b872c9acd4df480d0
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=fed13559459fd6177e688cb75f474130988d53aa077ea8d342ecf38ff50eb619
  Stored in directory: /Users/yuyingshan/Library/Caches/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built cufflinks retrying
Note: you may need to restart the kernel to use updated packages.


In [16]:
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [17]:
df['Product'].value_counts().sort_values(ascending=False).iplot(kind='bar', yTitle='Number of Complaints', 
                                                                title='Number complaints in each product')

In [19]:
# Text Pre-processing

def print_plot(index):
    example = df[df.index == index][['Consumer complaint narrative', 'Product']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Product:', example[1])
print_plot(10)

nan
Product: Credit reporting, credit repair services, or other personal consumer reports


In [20]:
print_plot(100)

nan
Product: Credit card or prepaid card


In [22]:
import re

In [25]:
from nltk.corpus import stopwords

In [74]:
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df['Consumer complaint narrative'] = df['Consumer complaint narrative'].apply(clean_text)
df['Consumer complaint narrative'] = df['Consumer complaint narrative'].str.replace('\d+', '')

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/yuyingshan/nltk_data'
    - '/Users/yuyingshan/opt/anaconda3/envs/AntonioEnv/nltk_data'
    - '/Users/yuyingshan/opt/anaconda3/envs/AntonioEnv/share/nltk_data'
    - '/Users/yuyingshan/opt/anaconda3/envs/AntonioEnv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
# LSTM Modeling

In [32]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [46]:
df=df.astype(str)

In [47]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['Consumer complaint narrative'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 164904 unique tokens.


In [50]:
import tensorflow as tf
import keras

In [53]:
from keras.preprocessing.sequence import pad_sequences

In [54]:
X = tokenizer.texts_to_sequences(df['Consumer complaint narrative'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (1867204, 250)


In [55]:
Y = pd.get_dummies(df['Product']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (1867204, 13)


In [39]:
pip install sklearn

  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=f37a5c16bd52ed505a4a85b75895a0ac56a190dbf75d29e242335b46e4859ca2
  Stored in directory: /Users/yuyingshan/Library/Caches/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.


In [42]:
from sklearn.model_selection import train_test_split

In [56]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1680483, 250) (1680483, 13)
(186721, 250) (186721, 13)


In [58]:
from keras.models import Sequential

In [60]:
from keras.layers import Embedding

In [63]:
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import LSTM

In [65]:
from keras.layers.core import Dense, SpatialDropout1D

In [67]:
from keras.callbacks import EarlyStopping

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1867204 entries, 0 to 1867203
Data columns (total 18 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   Date received                 object
 1   Product                       object
 2   Sub-product                   object
 3   Issue                         object
 4   Sub-issue                     object
 5   Consumer complaint narrative  object
 6   Company public response       object
 7   Company                       object
 8   State                         object
 9   ZIP code                      object
 10  Tags                          object
 11  Consumer consent provided?    object
 12  Submitted via                 object
 13  Date sent to company          object
 14  Company response to consumer  object
 15  Timely response?              object
 16  Consumer disputed?            object
 17  Complaint ID                  object
dtypes: object(18)
memory usage: 256.4+ MB


In [71]:
df.Product.value_counts()

Credit reporting, credit repair services, or other personal consumer reports    714020
Debt collection                                                                 324543
Mortgage                                                                        315552
Credit card or prepaid card                                                     185215
Bank account or service                                                          86206
Checking or savings account                                                      77414
Student loan                                                                     59916
Consumer Loan                                                                    31604
Vehicle loan or lease                                                            21300
Money transfer, virtual currency, or money service                               21198
Payday loan, title loan, or personal loan                                        21063
Money transfers                            

In [72]:
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2019-09-24,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,transworld systems inc. \nis trying to collect...,nan,TRANSWORLD SYSTEMS INC,FL,335XX,nan,Consent provided,Web,2019-09-24,Closed with explanation,Yes,nan,3384392
1,2019-09-19,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,nan,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,PA,15206,nan,Consent not provided,Web,2019-09-20,Closed with non-monetary relief,Yes,nan,3379500
2,2019-11-08,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,"Over the past 2 weeks, I have been receiving e...",nan,"Diversified Consultants, Inc.",NC,275XX,nan,Consent provided,Web,2019-11-08,Closed with explanation,Yes,nan,3433198
3,2019-09-15,Debt collection,Other debt,Attempts to collect debt not owed,Debt was result of identity theft,Pioneer has committed several federal violatio...,nan,Pioneer Capital Solutions Inc,CA,925XX,nan,Consent provided,Web,2019-09-15,Closed with explanation,Yes,nan,3374555
4,2019-05-20,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,nan,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,NC,28078,nan,Consent not provided,Web,2019-05-20,Closed with non-monetary relief,Yes,nan,3247381


In [75]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(13, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


/Users/yuyingshan/opt/anaconda3/envs/AntonioEnv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning:

Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.



Train on 1512434 samples, validate on 168049 samples
Epoch 1/5
 164992/1512434 [==>...........................] - ETA: 6:05:19 - loss: 1.6885 - accuracy: 0.4324

KeyboardInterrupt: 